In [2]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

import IPython
import IPython.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from load_data import merged_df

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

2.8.0


In [10]:
#load the dataset, slice a smaller amount for now
REDUCED_DATESET_SIZE = 100_000
df = merged_df()
df = df[:REDUCED_DATESET_SIZE]

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


2023-12-16 16:06:59.228825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-12-16 16:06:59.239235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [11]:
#test/train split
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

#drop the target column 
train_labels = train_dataset['target']
test_labels = test_dataset['target']

train_features = train_dataset.drop(columns=['target'])
test_features = test_dataset.drop(columns=['target'])

In [27]:
#create a normalization layer
normalizer = tf.keras.layers.Normalization(input_dim=23)

#normalize the train_features by subtracting the mean and dividing by the std deviation
normalizer.adapt(np.array(train_features))

2023-12-16 16:15:42.774215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-12-16 16:15:42.784949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [28]:
#build the three layer model with initial normalization layer
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

#fit the model
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_split=0.2,
    verbose=1, epochs=2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_7 (Normalizat  (None, 23)               47        
 ion)                                                            
                                                                 
 dense_9 (Dense)             (None, 64)                1536      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,808
Trainable params: 5,761
Non-trainable params: 47
_________________________________________________________________
Epoch 1/2
  12/2000 [..............................] - ETA: 9s - loss: 179.0008 

2023-12-16 16:15:50.187568: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1990/2000 [============================>.] - ETA: 0s - loss: 133.1772

2023-12-16 16:15:59.066790: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2000/2000 [==============================] - 10s 5ms/step - loss: 133.2131 - val_loss: 128.7805
Epoch 2/2
2000/2000 [==============================] - 10s 5ms/step - loss: 126.0407 - val_loss: 126.0836


In [29]:
# save the model to disk
import pickle
dnn_model_filename = '../models/dnn_model.sav'
pickle.dump(dnn_model, open(dnn_model_filename, 'wb'))

#save model using keras
dnn_model.save('../models/dnn_model.keras')

INFO:tensorflow:Assets written to: ram://aa73aaf0-d836-411a-bcdf-111ec5c126d5/assets
